In [4]:
from syncbn import SyncBatchNorm

import torch
from torch.utils.data import Dataset

In [5]:
sbn = SyncBatchNorm(5)

In [34]:
data = torch.zeros((4, 5))
data[0] = 1
data[3] = 1
data

tensor([[1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.]])

In [35]:
torch.sum(data, dim=0)


tensor([2., 2., 2., 2., 2.])

In [58]:
!python3 test_syncbn.py

^C


TypeError: cannot unpack non-iterable NoneType object

In [20]:
"""Non-blocking point-to-point communication."""
import time
import random
import torch.distributed as dist


def bn_test(rank, input_tensor):
    sbn = SyncBatchNorm(5)
    out = sbn(input_tensor)
        
    print('Rank ', rank, ' has out ', out)
    
    

def init_process(rank, size, fn, inp_tensor, master_port, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = str(master_port)
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, inp_tensor)
    
if __name__ == "__main__":
    size = 2
    data = torch.zeros((4, 5))
    processes = []
    port = random.randint(25000, 30000)
    for rank in range(size):
        ctx = torch.multiprocessing.get_context("spawn")
        p = ctx.Process(target=init_process, args=(rank, size, bn_test, data[rank:rank+2], port))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
  File "/usr/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'init_process' on <module '__main__' (built-in)>
    exitcode = _main(fd, parent_sentinel)
  File "/usr/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'init_process' on <module '__main__' (built-in)>


In [10]:
out = sbn(data)

RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.